In [79]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

In [80]:
dataset = pd.read_csv('../data/mt.csv')
X = pd.DataFrame(data=dataset, columns=['F1', 'F2']).to_numpy().astype(np.float64)
y1 = pd.DataFrame(data=dataset, columns=['T1']).to_numpy().astype(np.float64) - 1
y2 = pd.DataFrame(data=dataset, columns=['T2']).to_numpy().astype(np.float64)
y2 = y2 / np.max(y2, axis=0)

In [81]:
class Layer:
    def __init__(self, n_neurons_input, n_neurons_ouput):
        self.n_neurons_input = n_neurons_input
        self.n_neurons_ouput = n_neurons_ouput
        self.weights = np.random.randn(n_neurons_input, n_neurons_ouput)
        self.bias = np.random.randn(1, n_neurons_ouput)

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

    def backward(self, output_error, learning_rate):
        deriv_input = np.dot(output_error, self.weights.T)
        deriv_weights = np.dot(self.input.T, output_error)
        # bias_error = output_error
        
        self.weights -= learning_rate * deriv_weights
        self.bias -= learning_rate * output_error
        return deriv_input

In [82]:
class ActivationLayer:
    def __init__(self, activation, deriv_activ):
        self.activation = activation
        self.deriv_activ = deriv_activ
    
    def forward(self, input):
        self.input = input
        return self.activation(input)
    
    def backward(self, output_error, _):
        return (output_error * self.deriv_activ(self.input))

In [83]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / np.square(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.square(np.tanh(x))

In [84]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def deriv_mse(y_true, y_pred):
    return 2 * np.array((y_pred - y_true) / y_pred.size)

def binary_crossentropy(y_true, y_pred):
    eps = 1e-6
    return np.mean((y_true * np.log(y_pred + eps) + (1-y_true) * np.log(1-y_pred + eps)))

def deriv_bin_centropy(y_true, y_pred):
    return np.array((y_true - y_pred) / (y_pred * (1-y_pred)))

In [85]:
class Network:
    def __init__(self, learning_rate, epochs, batch_size):
        self.layers = []
        self.loss = None
        self.loss_prime = None
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs

    # add layer to network
    def add_layer(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        n_samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(n_samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward(output)
            result.append(output)

        result = np.array(result)
        # print(result.shape)
        # print(result)
        return [1 if o[0][0] > 0.5 else 0 for o in result], [o[0][1] for o in result]

    def stochastic_grad_descent(self, X_train, y1_train, y2_train):
        for i in range(0, X_train.shape[0], self.batch_size):
            X_batch = X_train[i:i+self.batch_size]
            y1_batch = y1_train[i:i+self.batch_size]
            y2_batch = y2_train[i:i+self.batch_size]

            for j in range(X_batch.shape[0]):
                err = 0
                # forward propagation
                output = X_batch[j].reshape(1, 2)

                for layer in self.layers:
                    output = layer.forward(output)

                # compute loss (for display purpose only)
                err_classify = self.loss(y1_batch[j], output)
                err_reg = self.loss(y2_batch[j], output)
                err += err_classify + err_reg

                # backward propagation
                error_classify = self.loss_prime(y1_batch[j], output)
                error_reg = self.loss_prime(y2_batch[j], output)
                error = error_classify + error_reg

                for layer in reversed(self.layers):
                    error = layer.backward(error, self.learning_rate)
                    
        return err_classify, err_reg

    # train the network
    def fit(self, X_train, y1_train, y2_train):
        # training loop
        for i in range(self.epochs):
            err_classify, err_reg = self.stochastic_grad_descent(
                X_train, y1_train, y2_train)

            # calculate average error on all samples
            print('epoch %d/%d  classify error=%f  regr error=%f' %
                  (i+1, self.epochs, err_classify, err_reg))

    def classify_accuracy(self, y_true, y_pred):
        return np.mean(y_pred == y_true)

    def regr_loss(self, y_true, y_pred):
        return mse(y_true, y_pred)


In [86]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2, random_state=42)

In [87]:
net = Network(learning_rate=0.1, epochs=100, batch_size=50)
net.add_layer(Layer(2, 2))
net.add_layer(ActivationLayer(tanh, tanh_prime))
net.add_layer(Layer(2, 2))
net.add_layer(ActivationLayer(sigmoid, sigmoid_prime))
net.add_layer(Layer(2, 2))

In [88]:
# train
net.use(mse, deriv_mse)
net.fit(X_train, y1_train, y2_train)

epoch 1/1000  classify error=0.229208  regr error=0.067904
epoch 2/1000  classify error=0.110616  regr error=0.020906
epoch 3/1000  classify error=0.039004  regr error=0.002077
epoch 4/1000  classify error=0.032085  regr error=0.001650
epoch 5/1000  classify error=0.031518  regr error=0.001745
epoch 6/1000  classify error=0.031474  regr error=0.001754
epoch 7/1000  classify error=0.031493  regr error=0.001749
epoch 8/1000  classify error=0.031525  regr error=0.001742
epoch 9/1000  classify error=0.031562  regr error=0.001733
epoch 10/1000  classify error=0.031602  regr error=0.001724
epoch 11/1000  classify error=0.031643  regr error=0.001714
epoch 12/1000  classify error=0.031685  regr error=0.001705
epoch 13/1000  classify error=0.031726  regr error=0.001695
epoch 14/1000  classify error=0.031768  regr error=0.001686
epoch 15/1000  classify error=0.031808  regr error=0.001676
epoch 16/1000  classify error=0.031847  regr error=0.001667
epoch 17/1000  classify error=0.031886  regr erro

In [89]:
# test
y_pred_classify, y_pred_reg = net.predict(X_test)

In [90]:
print(y_pred_classify)

[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]


In [91]:
y1_test[:, 0]

array([1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.])

In [92]:
print(y_pred_reg)

[0.7390378165173797, 0.04369552312959479, 0.8493254882598763, 0.768536881687347, 0.7718114319580354, 0.046626377986763945, 0.665190585617119, 0.8356523328891192, 0.768536881687347, 0.6720092536901324, 0.8250631154600819, -0.0008617503761019218, 0.005707964345145866, 0.0034565176912018103, 0.029773394798992214, 0.7880820354240343, 0.8406381399898736, 0.6466730561086876, 0.7413223272656511, 0.8386598351969156, 0.02113937402908994, 0.8100971847953342, 0.05449184986568123, 0.8360320318432239, 0.8418188182191297, 0.8368387725008968, 0.8290334334487204, 0.8434754733336592, 0.02365049955731391, 0.02113937402908994]


In [93]:
y2_test[:, 0]

array([0.35538752, 0.03213611, 1.        , 0.42533081, 0.42344045,
       0.03780718, 0.29489603, 0.73913043, 0.42533081, 0.29489603,
       0.64272212, 0.00882168, 0.01638311, 0.0094518 , 0.02835539,
       0.47385003, 0.80403277, 0.27032136, 0.36862004, 0.7763075 ,
       0.02016383, 0.5557656 , 0.04032766, 0.74102079, 0.80655325,
       0.75362319, 0.65784499, 0.85507246, 0.02646503, 0.02016383])

In [94]:
acc = net.classify_accuracy(y_pred_classify, y1_test[:, 0])
acc

1.0

In [95]:
loss = net.regr_loss(y_pred_reg, y2_test[:, 0])
loss

0.0448411822208525